# Setup

In [2]:
import os
import cv2
import json
import string
import random 
import gc
import csv
import pickle
import timm
import fractions
import numpy as np
from collections import Counter
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.optim import Adam
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import to_tensor
from torchvision.models.detection.image_list import ImageList
from transformers import DetrFeatureExtractor, DetrForObjectDetection
from torch.optim.lr_scheduler import ReduceLROnPlateau
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk.translate.bleu_score as bleu_score
from transformers import GPT2Tokenizer, GPT2LMHeadModel

data_dir = os.path.join('dataset')
# data_dir = os.path.join('dataset30K')
working_dir = os.path.join('working')
images_dir = os.path.join(data_dir,'Images')
captions_dir = os.path.join(data_dir,'captions.txt')

In [3]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU devices available.")
# *********************
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Device 0: NVIDIA GeForce GTX 1660 SUPER
Using device: cuda


In [4]:
# Load a pretrained Swin Transformer model
swin_model = timm.create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=0)  # num_classes=0 removes the classification head
swin_model.to(device)
swin_model.eval()

# # Freeze the Swin model parameters
# for param in swin_model.parameters():
#     param.requires_grad = False

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=192, out_features=576, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=192, out_features=192, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=192, out_features=768, bias=True)
            (act): GELU(approximate='none')
            (

In [5]:
# # Load a pretrained Swin Transformer model
# swin_model = timm.create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=0)  # num_classes=0 removes the classification head
# swin_model.to(device)
# swin_model.train()  # Set to training mode

# # Optionally, you might still want to freeze some layers if you don't want to train the entire model
# # Example: Freezing earlier layers but training later layers
# for name, param in swin_model.named_parameters():
#     if 'stage4' in name:  # Just an example, adjust according to your needs
#         param.requires_grad = True
#     else:
#         param.requires_grad = False


In [6]:
# Load pretrained DETR model and feature extractor
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50")
detr_model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50").to(device)

# Set model to evaluation mode
detr_model.eval()


C:\Users\Mohamed\anaconda3\Lib\site-packages\transformers\models\detr\feature_extraction_detr.py:38: FutureWarning: The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForO

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [7]:
# Initialize GPT tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # You can use 'gpt2-medium' or 'gpt2-large' for larger models
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')

# # Resize GPT's token embedding if you plan to add special tokens like <PAD> or <BOS>
# tokenizer.add_special_tokens({'pad_token': '<PAD>'})


C:\Users\Mohamed\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Preprocess & encode captions

In [9]:
# for flickr8k
def load_captions(filepath):
    captions = {}
    with open(filepath, 'r', encoding='utf-8') as file:
        reader = csv.reader(file) 
        for row in reader:
            if len(row) != 2:
                print(f"Skipping malformed line: {row[:50]}...")  
                continue
            image_id, caption = row
            image_id = image_id.split('.')[0]  
            if image_id not in captions:
                captions[image_id] = []
            captions[image_id].append(caption)
    
    return captions
captions = load_captions(captions_dir)

In [10]:
def preprocess_caption(caption, tokenizer, max_length):
    # Tokenize caption using GPT2 tokenizer
    inputs = tokenizer(caption, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt")
    return inputs.input_ids, inputs.attention_mask

In [11]:
# def clean_and_tokenize(caption):
#     tokens = caption.lower().translate(str.maketrans('', '', string.punctuation)).split()
#     return tokens
    
# # Collect all captions
# all_captions = []
# for cap_list in captions.values():
#     all_captions.extend(cap_list)
    
# # Count word frequencies
# word_counts = Counter()
# for caption in all_captions:
#     word_counts.update(clean_and_tokenize(caption))

# # Create a vocabulary with words that exist in both Word2Vec and your dataset
# vocab = [word for word, count in word_counts.items() if count >= 2]

# # Map words to indices for the special tokens
# word_to_ix = {word: ix for ix, word in enumerate(vocab, start=4)}  # start=4 to leave 0 for <PAD>, 1 for <START>, 2 for <END>, 3 for <UNK>
# word_to_ix['<PAD>'] = 0
# word_to_ix['<START>'] = 1
# word_to_ix['<END>'] = 2
# word_to_ix['<UNK>'] = 3

# # Reverse lookup for decoding
# ix_to_word = {ix: word for word, ix in word_to_ix.items()}

# # Update vocab size
# vocab_size = len(word_to_ix)
# vocab_size

In [12]:
# vocab_file_path = 'vocab.json'
# with open('vocab.json', 'w') as vocab_file:
#     json.dump(word_to_ix, vocab_file)

In [13]:
# def encode_caption(caption, word_to_ix, max_length):
#     tokens = clean_and_tokenize(caption)
#     tokens = ['<START>'] + tokens + ['<END>']
#     caption_ids = [word_to_ix.get(token, word_to_ix['<UNK>']) for token in tokens]
#     if len(caption_ids) < max_length:
#         caption_ids += [word_to_ix['<PAD>']] * (max_length - len(caption_ids))
#     else:
#         caption_ids = caption_ids[:max_length]
#     return np.array(caption_ids)

# max_length = max(len(clean_and_tokenize(caption)) + 2 for caption in all_captions)  # +2 for <START> and <END>
# encoded_captions = {img_id: [encode_caption(caption, word_to_ix, max_length) for caption in cap_list]
#                     for img_id, cap_list in captions.items()}


# Features extraction

In [15]:
def extract_swin_features(swin_model, image_path, transform):
    # Preprocess the image
    image = Image.open(image_path).convert("RGB") 
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    image_tensor = image_tensor.to(device)
    # Extract features
    with torch.no_grad():
        features = swin_model(image_tensor)
    
    # Swin transformer returns [batch_size, num_patches, embedding_dim]
    # You can reshape or pool as needed for your LSTM input
    return features.view(features.size(0), -1)  # Flatten into [batch_size, feature_dim]

def extract_detr_features(detr_model, image_path, transform, target_size=1536):
    # Load image and convert to RGB
    image = Image.open(image_path).convert("RGB")
    image = image.resize((224, 224))  # Adjust the size as needed
    
    # Apply the transformation (resize, normalize, etc.) to the image
    inputs = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

    with torch.no_grad():
        # Get outputs from the DETR model
        outputs = detr_model(inputs)

    # Extract object queries (decoder hidden states)
    object_queries = outputs.last_hidden_state  # Output of the last decoder layer

    # Flatten the object queries into a 2D tensor [batch_size, num_queries * hidden_dim]
    flattened_object_queries = object_queries.view(object_queries.size(0), -1)

    # If needed, reduce dimensionality via mean pooling to match target size (1536)
    if flattened_object_queries.size(1) > target_size:
        pooled_object_queries = torch.nn.functional.adaptive_avg_pool1d(
            flattened_object_queries.unsqueeze(0), target_size).squeeze(0)
    else:
        pooled_object_queries = flattened_object_queries

    return pooled_object_queries.cpu()

In [16]:
# List of image IDs (filenames without extension)
image_ids = [img_name.split('.')[0] for img_name in os.listdir(images_dir) if img_name.endswith('.jpg')]

# Set up transforms for training and validation/test
train_transform = transforms.Compose([
    transforms.RandomResizedCrop((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Define the transform used during feature extraction (should be fixed)
feature_extraction_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [17]:
# Load features from a .pkl file or extract them if not present
if data_dir == 'dataset':
    features_file = 'featuresMixed.pkl'
    print('using flickr8K dataset')
elif data_dir == 'dataset30K':
    features_file = 'featuresMixed30K.pkl'
    print('using flickr30K dataset')
else:
    raise ValueError(f"Unrecognized dataset")
    
features_dict = {}
if os.path.exists(features_file):
    with open(features_file, 'rb') as f:
        features_dict = pickle.load(f)
else:
    # Extract features for each image and save them
    for image_id in tqdm(image_ids):
        image_path = os.path.join(images_dir, image_id + '.jpg')
        if os.path.exists(image_path):
            swin_features = extract_swin_features(swin_model,image_path, feature_extraction_transform)
            detr_features = extract_detr_features(detr_model,image_path, feature_extraction_transform, target_size=1536)
            features_dict[image_id] = (swin_features, detr_features)

    with open(features_file, 'wb') as f:
        pickle.dump(features_dict, f)

using flickr8K dataset


# Model building

In [19]:
class Flickr8kDataset(Dataset):
    def __init__(self, features_dict, captions, tokenizer, image_ids, transform):
        """
        Args:
            features_dict (dict): Dictionary mapping image IDs to feature tuples (swin_features, detr_features).
            captions (list): List of captions for images.
            image_ids (list): List of image IDs.
            transform (callable, optional): Optional transform to be applied to the features.
        """
        self.features_dict = features_dict 
        self.captions = captions
        self.image_ids = image_ids
        self.transform = transform
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        
        # Retrieve swin_features and detr_features from the dictionary
        swin_features, detr_features = self.features_dict[image_id]

        # Apply clone and detach to each feature separately
        swin_features = swin_features.clone().detach().float().to(device)
        detr_features = detr_features.clone().detach().float().to(device)

        # Flatten both features to 1D
        swin_features_flat = swin_features.flatten()  # Shape: [1536]
        detr_features_flat = detr_features.flatten()  # Shape: [25600]
        
        # Combine features
        features = torch.cat([swin_features, detr_features], dim=1)
        
        # Retrieve all captions for this image
        captions_for_image = self.captions[idx]
        
        # Tokenize all captions for this image
        input_ids_list = []
        attention_mask_list = []
        
        for caption in captions_for_image:
            encoding = tokenizer(caption, return_tensors="pt", padding="max_length", truncation=True, max_length=38)
            input_ids_list.append(encoding['input_ids'].squeeze(0))  # Squeeze batch dimension
            attention_mask_list.append(encoding['attention_mask'].squeeze(0))  # Squeeze batch dimension
        
        input_ids = torch.stack(input_ids_list)  # Stack all tokenized captions
        attention_mask = torch.stack(attention_mask_list)  # Stack all attention masks
        
        return features, input_ids, attention_mask


In [20]:
def custom_collate_fn(batch):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Filter out None values from the batch
    batch = list(filter(lambda x: x is not None, batch))
    
    # If the batch is empty, return empty tensors
    if len(batch) == 0:
        return torch.tensor([]).to(device), torch.tensor([]).to(device), torch.tensor([]).to(device)
    
    # Unzip the batch into features, input_ids, and attention_mask
    features, input_ids, attention_masks = zip(*batch)
    
    # Determine the maximum feature size in the batch
    max_feature_size = max(feature.size(0) for feature in features)
    
    # Pad feature tensors to the maximum size
    features = torch.stack([
        torch.cat((feature.to(device), torch.zeros((max_feature_size - feature.size(0),), device=device)), dim=0)
        for feature in features
    ])
    
    # Determine the maximum caption length in the batch for input_ids and attention_masks
    max_caption_length = max(ids.size(0) for ids in input_ids)
    
    # Pad input_ids to the maximum length
    input_ids_padded = torch.stack([
        torch.cat((ids.to(device), torch.zeros((max_caption_length - ids.size(0),), dtype=torch.long, device=device)), dim=0)
        for ids in input_ids
    ])
    
    # Pad attention_mask to the maximum length
    attention_mask_padded = torch.stack([
        torch.cat((mask.to(device), torch.zeros((max_caption_length - mask.size(0),), dtype=torch.long, device=device)), dim=0)
        for mask in attention_masks
    ])
    
    return features, input_ids_padded, attention_mask_padded
    
print(type(captions))  # This should print <class 'dict'>

<class 'dict'>


In [21]:
# Shuffle and split data
image_ids = list(captions.keys())
random.seed(42)
random.shuffle(image_ids)

# Calculate indices for splits
total_images = len(image_ids)
train_end = int(0.7 * total_images)
val_end = int(0.9 * total_images)

train_ids = image_ids[:train_end]
val_ids = image_ids[train_end:val_end]
test_ids = image_ids[val_end:]

In [22]:
train_dataset = Flickr8kDataset(features_dict, captions, tokenizer, train_ids, train_transform)
val_dataset = Flickr8kDataset(features_dict, captions, tokenizer, val_ids, test_transform)
test_dataset = Flickr8kDataset(features_dict, captions, tokenizer, test_ids, test_transform)

# Create DataLoaders for each split
batch_size= 16
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, collate_fn=custom_collate_fn)

In [23]:
sample_image_path = "dataset/Images/133905560_9d012b47f3.jpg"  # Replace with a valid image path
swin_features = extract_swin_features(swin_model, sample_image_path, feature_extraction_transform).to(device)
detr_features = extract_detr_features(detr_model, sample_image_path, feature_extraction_transform, target_size=1536).to(device)

# Combine features (Swin + DETR) - you can concatenate or average them
combined_features  = torch.cat([swin_features.flatten(), detr_features.flatten()], dim=0)

# Get the feature size as an integer
feature_size = combined_features.size(0)

# Print feature size
print("Swin Features Size:", swin_features.size())
print("DETR Features Size:", detr_features.size())
print("Combined Feature Size:", feature_size)
# feature_size = sample_features.size(1)
# print("Feature Size:", feature_size)

Swin Features Size: torch.Size([1, 1536])
DETR Features Size: torch.Size([1, 1536])
Combined Feature Size: 3072


In [24]:
class ImageCaptioningModel(nn.Module):
    def __init__(self, feature_size, gpt_model, embed_size, dropout):
        super(ImageCaptioningModel, self).__init__()
        self.gpt_model = gpt_model
        self.dropout = nn.Dropout(dropout)
        self.embed_size = self.gpt_model.config.n_embd
        self.feature_fc = nn.Linear(feature_size, embed_size)  # Adapting Swin output to LSTM
    
    def forward(self, features, captions, attention_mask=None):
        """
        Args:
            features: Image features extracted (e.g., from Swin and DETR), size: [batch_size, feature_size]
            input_ids: Tokenized captions (GPT-2 token IDs), size: [batch_size, seq_length]
            attention_mask: Attention mask for GPT-2, size: [batch_size, seq_length]
        """
        features = features.to(device)
        captions = captions.to(device)
        
        # Project features to embedding size
        batch_size = features.size(0)
        features = self.feature_fc(features)  # Should be of shape [batch_size, embed_size]
        features = features.unsqueeze(1)# Add a sequence dimension: [batch_size, 1, embed_size]

        # Project features to GPT-2's embedding size
        features = self.feature_fc(features)  # Shape: [batch_size, gpt2_embedding_size]
        features = features.unsqueeze(1)  # Add a sequence dimension: [batch_size, 1, gpt2_embedding_size]

        # Concatenate image features with the input_ids (caption tokens)
        # The feature vector is treated as the initial input for GPT-2, followed by the tokenized caption
        embeddings = self.gpt2.transformer.wte(input_ids)  # Get word embeddings for input_ids
        inputs_embeds = torch.cat((features, embeddings), dim=1)  # Concatenate along sequence dimension

        # Pass through GPT-2
        outputs = self.gpt2(inputs_embeds=inputs_embeds, attention_mask=attention_mask)

        return outputs.logits  # Return logits for next-token prediction




hidden_size = 1024
dropout = 0.3
captioning_model = ImageCaptioningModel(feature_size, hidden_size, vocab_size, embed_size, dropout)
captioning_model = captioning_model.to(device)

NameError: name 'vocab_size' is not defined

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0.01):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Training & Validation

In [ ]:
# Training loop
criterion = nn.CrossEntropyLoss(ignore_index=word_to_ix['<PAD>'])
optimizer = Adam(captioning_model.parameters(), lr=0.004, weight_decay=0.0001)
# Add a StepLR scheduler - decays the learning rate by gamma every 'step_size' epochs
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, threshold=0.01, min_lr=1e-6)
num_epochs = 60
early_stopping = EarlyStopping(patience=4, delta=0.01)  # Adjust patience and delta as needed
train_loss_values = []
val_loss_values = []

# Create a tqdm instance to show global progress for training and validation
total_steps = num_epochs * len(train_loader) + num_epochs * len(val_loader)
with tqdm(total=total_steps, desc='Training Progress') as pbar:
    for epoch in range(num_epochs):
        captioning_model.train()
        epoch_train_loss = 0
        for i, (features, captions) in enumerate(train_loader):
            if features.shape[0] == 0:  # Skip if batch is empty
                continue
                
            # Move inputs and targets to the selected device (GPU or CPU)
            features = features.to(device)
            captions = captions.to(device)
            
            optimizer.zero_grad()

            # Get the size of the captions
            batch_size, num_captions, seq_len = captions.size()

            # Squeeze out the extra dimensions in `features` to make it [batch_size, feature_size]
            features = features.squeeze(1).squeeze(1)  # Removing the unnecessary singleton dimensions

            # Now expand the features to repeat them across the captions dimension
            features = features.unsqueeze(1).expand(-1, num_captions, -1)  # [batch_size, num_captions, feature_size]

            # Flatten features and captions for input to the model
            features = features.contiguous().view(batch_size * num_captions, -1)
            captions = captions.view(batch_size * num_captions, seq_len)

            # Forward pass
            outputs = captioning_model(features, captions)
            loss = criterion(outputs[:, :captions.size(1), :].view(-1, vocab_size), captions.view(-1))

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            epoch_train_loss += loss.item()

            pbar.set_postfix({'Step': i, 'Loss': loss.item()})
            pbar.update(1)
        
        epoch_train_loss /= len(train_loader)
        train_loss_values.append(epoch_train_loss)
       
        # Validation loop
        captioning_model.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for features, captions in val_loader:
                if features.shape[0] == 0:  # Skip empty batches
                    continue
                # Move inputs and targets to the selected device (GPU or CPU)
                features = features.to(device)
                captions = captions.to(device)
                
                # Get the size of the captions
                batch_size, num_captions, seq_len = captions.size()

                # Squeeze and expand features just like in the training loop
                features = features.squeeze(1).squeeze(1)  # Remove unnecessary singleton dimensions
                features = features.unsqueeze(1).expand(-1, num_captions, -1)  # Expand to match num_captions
                features = features.contiguous().view(batch_size * num_captions, -1)

                captions = captions.view(batch_size * num_captions, seq_len)

                # Forward pass
                outputs = captioning_model(features, captions)
                loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
                epoch_val_loss += loss.item()

        epoch_val_loss /= len(val_loader)
        val_loss_values.append(epoch_val_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}],  ' f'Training Loss: {epoch_train_loss:.3f},  ' f'Validation Loss: {epoch_val_loss:.3f}')
        
        # Update the learning rate scheduler based on validation loss
        scheduler.step(epoch_val_loss)

        # Early stopping
        early_stopping(epoch_val_loss)
        if early_stopping.early_stop:
            print("Early stopping")
            break

In [ ]:
model_save_path = 'captioning_model.pth'

# Save the model's state dictionary
torch.save(captioning_model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# Define the path where the model is saved
model_save_path = 'captioning_model.pth'

# Load the state dictionary into the model
captioning_model.load_state_dict(torch.load(model_save_path))
# Set the model to evaluation mode
captioning_model.eval()

print(f"Model loaded from {model_save_path}")

In [ ]:
# Save loss values to a file
with open('train_loss_values.json', 'w') as f:
    json.dump(train_loss_values, f)
# Save loss values to a file
with open('val_loss_values.json', 'w') as f:
    json.dump(val_loss_values, f)

In [ ]:
def load_json(filename):
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        if not data:
            print(f"Warning: {filename} is empty.")
        return data
    except FileNotFoundError:
        print(f"Error: {filename} not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {filename}.")
        return None

# Load the loss values
train_loss_values = load_json('train_loss_values.json')
val_loss_values = load_json('val_loss_values.json')

# show thier values
train_loss_values ,val_loss_values

# Simplified plot without specifying figsize
plt.plot(train_loss_values, label='Training Loss')
plt.plot(val_loss_values, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Testing phase
captioning_model.eval()
test_loss = 0
with torch.no_grad():
    for features, captions in test_loader:
        if features.shape[0] == 0:  # Skip empty batches
            continue
        # Move inputs and targets to the selected device (GPU or CPU)
        features = features.to(device)
        captions = captions.to(device)
                
        # Get the size of the captions
        batch_size, num_captions, seq_len = captions.size()

        # Squeeze and expand features like in training and validation loops
        features = features.squeeze(1).squeeze(1)  # Remove unnecessary singleton dimensions
        features = features.unsqueeze(1).expand(-1, num_captions, -1)  # Expand to match num_captions
        features = features.contiguous().view(batch_size * num_captions, -1)

        captions = captions.view(batch_size * num_captions, seq_len)

        # Forward pass
        outputs = captioning_model(features, captions)
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        test_loss += loss.item()

# Compute average test loss
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss}')


# Inference

In [ ]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = feature_extraction_transform(image).unsqueeze(0)  # Add batch dimension
    return image
    
def extract_swin_image_features(model, image_path):
    image_tensor = preprocess_image(image_path).to(device)
    with torch.no_grad():
        features = swin_model(image_tensor)
    return features.view(features.size(0), -1)

def extract_detr_image_features(model, image_path, target_size=1536):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((224, 224))

    inputs = feature_extractor(images=image, return_tensors="pt").to(device)
    pixel_values = inputs['pixel_values']
    
    with torch.no_grad():
        outputs = model(pixel_values)
    object_queries = outputs.last_hidden_state 
    
    flattened_object_queries = object_queries.view(object_queries.size(0), -1) 

    # If needed, reduce dimensionality via mean pooling to match target size (1536)
    if flattened_object_queries.size(1) > target_size:
        pooled_object_queries = torch.nn.functional.adaptive_avg_pool1d(
            flattened_object_queries.unsqueeze(0), target_size).squeeze(0)
    else:
        pooled_object_queries = flattened_object_queries

    return pooled_object_queries.cpu()

In [ ]:
def remove_repetitive_words(caption):
    words = caption.split()
    filtered_words = [words[0]]  # Initialize with the first word
    
    for i in range(1, len(words)):
        if words[i] != words[i - 1]:
            filtered_words.append(words[i])
    
    return ' '.join(filtered_words)

def generate_caption(captioning_model, swin_model, detr_model, image_path, word_to_ix, ix_to_word, max_length, feature_size, beam_size=5):
    """
    Generate a caption for an image using the trained captioning model and Swin Transformer for feature extraction.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Preprocess the image to extract features
    image = Image.open(image_path).convert("RGB")
    image = feature_extraction_transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

    with torch.no_grad():
         # Extract features from Swin Transformer
        swin_features = extract_swin_image_features(swin_model, image_path).to(device)  # Swin features

        # Extract features from DETR
        detr_features = extract_detr_image_features(detr_model, image_path, target_size=1536).to(device)  # DETR features

        # Combine features from both models
        features = torch.cat([swin_features, detr_features], dim=1)  # Concatenate along the feature dimension

    # Initialize beam search
    beam = [([word_to_ix['<START>']], 0)]  # (sequence, score)
    for _ in range(max_length):
        new_beam = []
        for seq, score in beam:
            caption_tensor = torch.tensor(seq, dtype=torch.long).unsqueeze(0).to(device)  # Move to device
            with torch.no_grad():
                outputs = captioning_model(features, caption_tensor)
            
            # Get top beam_size predictions
            top_k_scores, top_k_ids = torch.topk(outputs[0, -1], beam_size)
            
            for i in range(beam_size):
                new_seq = seq + [top_k_ids[i].item()]
                new_score = score + top_k_scores[i].item()
                new_beam.append((new_seq, new_score))

        # Keep only the top beam_size sequences
        beam = sorted(new_beam, key=lambda x: x[1], reverse=True)[:beam_size]
        
        # Check for end token
        if any(seq[-1] == word_to_ix['<END>'] for seq, _ in beam):
            break

    # Choose the best sequence
    best_seq = max(beam, key=lambda x: x[1])[0]

    # Skip <START> and <END> tokens
    caption = ' '.join([ix_to_word[ix] for ix in best_seq if ix not in [word_to_ix['<START>'], word_to_ix['<END>']]])

    # Remove repetitive words from the caption
    caption = remove_repetitive_words(caption)
    
    return caption

In [ ]:
# Function to randomly select 5 images from the directory
def get_random_images(image_dir, num_images=5):
    all_images = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('jpg', 'jpeg', 'png'))]
    return random.sample(all_images, num_images)

# Function to generate caption and plot the image
def generate_and_plot_image(image_path, captioning_model, swin_model, detr_model, word_to_ix, ix_to_word, max_length=30, feature_size=3072):
    try:
        # Synchronize CUDA (if available)
        torch.cuda.synchronize()

        # Load and process the image
        imagez = Image.open(image_path).convert('RGB')
        
        # Generate caption using preloaded models
        caption = generate_caption(captioning_model, swin_model, detr_model, image_path, word_to_ix, ix_to_word, max_length, feature_size, beam_size=3)
        
        # Convert image to numpy array
        image_array = np.array(imagez)
        
        # Create a new figure
        fig, ax = plt.subplots()
        
        # Plot the image and add the caption
        ax.imshow(image_array)
        ax.axis('off')  # Hide axes
        ax.set_title(caption)
        
        # Save the image with caption
        save_path = os.path.splitext(image_path)[0] + '_with_caption.png'
        fig.savefig(save_path, bbox_inches='tight', pad_inches=0)  # Save the plot as a PNG file
        
        # Display the image and the caption
        plt.show()  # Ensure the image is displayed with the caption
        
        # Clear the figure
        plt.close(fig)  # Close the figure after displaying

        # Free resources by clearing the GPU cache and forcing garbage collection
        torch.cuda.empty_cache()  # Clear the GPU cache
        gc.collect()  # Force garbage collection to free RAM

    except Exception as e:
        print(f"Error during caption generation for {image_path}: {e}")

In [ ]:
# Main function to generate captions for 5 random images
def caption_random_images(images_dir, captioning_model, swin_model, detr_model, word_to_ix, ix_to_word):
    random_images = get_random_images(images_dir, num_images=5)
    
    for image_path in random_images:
        print(f"Generating caption for: {image_path}")
        generate_and_plot_image(image_path, captioning_model, swin_model, detr_model, word_to_ix, ix_to_word)

# Call the function to generate captions for 5 random images
caption_random_images(images_dir, captioning_model, swin_model, detr_model, word_to_ix, ix_to_word)

# Metrics scores

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

In [ ]:
# Custom fraction class to avoid the unexpected keyword argument '_normalize' issue
class CustomFraction(fractions.Fraction):
    def __new__(cls, numerator=0, denominator=None, _normalize=True):
        return super().__new__(cls, numerator, denominator)

# Override the modified_precision function in nltk
bleu_score.Fraction = CustomFraction

# Testing phase with BLEU score calculation and tqdm progress bar
captioning_model.eval()
test_loss = 0
bleu_scores = []
smoothing_function = SmoothingFunction().method4

# Lists to collect actual and predicted captions for BLEU score calculation
actual_captions = []
predicted_captions_list = []

# Use tqdm to track the progress of the testing phase
with torch.no_grad():
    for features, captions in tqdm(test_loader, desc="Testing"):
        if features.shape[0] == 0:
            continue
        features = features.to(device)
        captions = captions.to(device)
        # Get the size of the captions
        batch_size, num_captions, seq_len = captions.size()

        # Squeeze and expand features like in training and validation loops
        features = features.squeeze(1).squeeze(1)  # Remove unnecessary singleton dimensions
        features = features.unsqueeze(1).expand(-1, num_captions, -1)  # Expand to match num_captions
        features = features.contiguous().view(batch_size * num_captions, -1)

        captions = captions.view(batch_size * num_captions, seq_len)

        # Forward pass
        outputs = captioning_model(features, captions)
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        test_loss += loss.item()

        # Decode the outputs and collect captions
        predicted_captions = outputs.argmax(2).cpu().numpy()
        for pred, actual in zip(predicted_captions, captions.cpu().numpy()):
            pred_caption = [ix_to_word[ix] for ix in pred if ix not in {word_to_ix['<PAD>'], word_to_ix['<START>'], word_to_ix['<END>'], word_to_ix['<UNK>']}]
            actual_caption = [ix_to_word[ix] for ix in actual if ix not in {word_to_ix['<PAD>'], word_to_ix['<START>'], word_to_ix['<END>'], word_to_ix['<UNK>']}]
            actual_captions.append(actual_caption)
            predicted_captions_list.append(pred_caption)

# Calculate BLEU scores for all captions
bleu1 = corpus_bleu([[actual] for actual in actual_captions], predicted_captions_list, weights=(1, 0, 0, 0), smoothing_function=smoothing_function)
bleu2 = corpus_bleu([[actual] for actual in actual_captions], predicted_captions_list, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing_function)
bleu3 = corpus_bleu([[actual] for actual in actual_captions], predicted_captions_list, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothing_function)
bleu4 = corpus_bleu([[actual] for actual in actual_captions], predicted_captions_list, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing_function)

# Calculate average loss
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss}')
print(f'BLEU-1 Score: {bleu1}')
print(f'BLEU-2 Score: {bleu2}')
print(f'BLEU-3 Score: {bleu3}')
print(f'BLEU-4 Score: {bleu4}')

# Average bleu score
bleu_scores = [bleu1, bleu2, bleu3, bleu4]
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f'Average BLEU Score: {average_bleu_score}')